In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 

In [2]:
#Read data of sensors
sensorData = pd.read_excel('Data/Environement/K=3/SensorInformation.xls')
sensorCoordinate = []

In [3]:
#Sensors coordinate 
for row in sensorData.index:
    coordinate = (sensorData['X'][row] ,sensorData['Y'][row] )
    sensorCoordinate.append(coordinate)

In [4]:
#Cluster Information 
clusterData = pd.read_excel('Data/Environement/K=3/ClustersInformations.xls')
ClusterInformation = []

In [5]:
#Cluster coordinate 
for row in clusterData.index:
    coordinate = (clusterData['X'][row] ,clusterData['Y'][row] )
    ClusterInformation.append(coordinate)


In [6]:
# Normlized Data of the Mission 
nomlizedMissionData = pd.read_excel("NormlizedData.xls")
missionInformationSubPath = []
missionInformationScores =  []
for row in nomlizedMissionData.index:
    subPath = (nomlizedMissionData['Cluster_ID'][row] , nomlizedMissionData['Cluster_DESTINATION'][row])
    scores  =(float(nomlizedMissionData['TIME_TRAVEL_CONSUMPTION'][row]),float(nomlizedMissionData['HOVERING_TIME'][row]) , float(nomlizedMissionData['ENERGY_HOVER'][row])  , float(nomlizedMissionData['TRAVEL_ENERGY_COST'][row]))
    missionInformationSubPath.append(subPath)
    missionInformationScores.append(scores)

In [7]:
# Starting with gurobi , Match subpath with there scores 
import gurobipy as gp 
from gurobipy import GRB 

In [8]:
dataDict = {}
for cluster in range(len(missionInformationSubPath)):
    dataDict.update({missionInformationSubPath[cluster] : missionInformationScores[cluster]})

In [9]:
#Match subpath with there scores  
combinations , timeTravel , hoveringTime , energyTravel , hoveringEnergy  = gp.multidict(dataDict)

In [10]:
numberOfCluster = 14

In [11]:
# The decision variable of the subPath it's equal to 1 if the cluster i is connected with the cluster j 
#Initialization of the model 
model  = gp.Model('UAV_TimeEnergy_Minimization')

Restricted license - for non-production use only - expires 2023-10-25


In [53]:
# Create the decision variable for the UAV_TimeEnergy_Minimization 
# combinations contain the cluster i and the cluster j i--j or a subPath 
decisionVarForSubPath = model.addVars(combinations , vtype = GRB.BINARY , name='SubPath')

In [54]:
# Write some constraint 
#-----------------------------------------------------
#The startingPosition Should be the base station  (C1) --> keep  
startingPosition = 0 
# The number  sensor in a cluster should be at least = 1 or greater  (C2) --> keep   
# The cluster should be in the zone covered by the UAV  (C3) --> keep   
# a visited cluster should not be revisited by the UAV (C4) --> keep 
# a deadline for a sensor to send th data 
# The energy consumption should be less than  a certain value where the UAV 
# a feaseble solution should satisfy all the constraints 
visitedCluster = {}

In [150]:
import random 
Lambda  = random.uniform(0,1)

In [151]:
print("λ : ",Lambda , '|1-λ' , 1-Lambda)

λ :  0.5856585075152124 |1-λ 0.4143414924847876


In [154]:
#Constraint C1 :
listOfCluster = []
clusterIndex=0 
for element in range(len(decisionVarForSubPath)): 
    listOfCluster.append(clusterIndex)
    clusterIndex+=1
visitedCluster = {3}
#c1 = model.addConstr((decisionVarForSubPath.==path[0]) , name='Starting Position Constraint')

In [189]:
#Define the objective function
import random 
from gurobipy import quicksum 
λ1  = random.uniform(0,1)
λ2  = 1 - λ1  
model.NumObj = 2 
model.modelSense = GRB.MINIMIZE
model.setObjectiveN(quicksum(timeTravel[i,j] + hoveringTime[i,j] for  i,j in decisionVarForSubPath) , index=0 , weight = λ1)
model.setObjectiveN(quicksum(energyTravel[i,j]+hoveringEnergy[i,j] for i,j in decisionVarForSubPath) , index=1 , weight = λ2)

In [196]:
#C1 
path={}
C1 = model.addConstr(quicksum(decisionVarForSubPath[i,j] for j in v if j!=i )==1 for i in decisionVarForSubPath)

KeyError: 0

In [199]:
for i in decisionVarForSubPath:
    for j in decisionVarForSubPath:
        if i!=j:
            print(j)

(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(1, 0)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(1, 12)
(2, 0)
(2, 1)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(2, 8)
(2, 9)
(2, 10)
(2, 11)
(2, 12)
(3, 0)
(3, 1)
(3, 2)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)
(3, 9)
(3, 10)
(3, 11)
(3, 12)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 5)
(4, 6)
(4, 7)
(4, 8)
(4, 9)
(4, 10)
(4, 11)
(4, 12)
(5, 0)
(5, 1)
(5, 2)
(5, 3)
(5, 4)
(5, 6)
(5, 7)
(5, 8)
(5, 9)
(5, 10)
(5, 11)
(5, 12)
(6, 0)
(6, 1)
(6, 2)
(6, 3)
(6, 4)
(6, 5)
(6, 7)
(6, 8)
(6, 9)
(6, 10)
(6, 11)
(6, 12)
(7, 0)
(7, 1)
(7, 2)
(7, 3)
(7, 4)
(7, 5)
(7, 6)
(7, 8)
(7, 9)
(7, 10)
(7, 11)
(7, 12)
(8, 0)
(8, 1)
(8, 2)
(8, 3)
(8, 4)
(8, 5)
(8, 6)
(8, 7)
(8, 9)
(8, 10)
(8, 11)
(8, 12)
(9, 0)
(9, 1)
(9, 2)
(9, 3)
(9, 4)
(9, 5)
(9, 6)
(9, 7)
(9, 8)
(9, 10)
(9, 11)
(9, 12)
(10, 0)
(10, 1)
(10, 2)
(10, 3)
(10, 4)
(10, 5)
(10, 6)
(10, 7)
(10, 8)
(10, 9)
(10, 11)
(10, 12)
(11, 0)
(11, 1)
(11, 2)
(11, 3)
(11, 4)